## Stockholms stads offentliga toaletter
* [#1](https://github.com/salgo60/Offentliga-toaletter/issues/1)
   * karta [Toaletter_sthlm_latest.html](https://salgo60.github.io/Offentliga-toaletter/Notebook/Toaletter_sthlm_latest.html)
* Notebook [Toaletter_1_Sthlm.ipynb](https://github.com/salgo60/Offentliga-toaletter/blob/main/Notebook/Toaletter_1_Sthlm.ipynb) 

---- 
* 0.1 skapade
* 0.2 la till länkar Googfle Map etc... i popup
* 0.3 söker fram toaletter i OSM i dag
* 0.4 visar OSM egenskaper och bild wikicommons
* 0.5 visar även OSM disused:amenity toilets
* 0.6 skapar ett jcdecaux_layer
* 0.7 försökt matcha appen Toa Sverige med OSM data se 


In [1]:
import time
from datetime import datetime 
import glob
import pandas as pd
from datetime import date

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))



Start: 2026-01-30 02:04:39


In [2]:
#used by version 0.3 
SthlmfileName =f"data/raw/Toalett_Punkt_Sthlm_20260123.gpkg"
Mapversion = "0.7"

# used by part 2
#output_path = f"data/raw/Toalett_Punkt_Sthlm_{today}.gpkg"
output_path = f"data/raw/Toalett_Punkt_Sthlm_20260124.gpkg"

In [3]:
import geopandas as gpd

gdf_raw = gpd.read_file(output_path)

print("Antal objekt:", len(gdf_raw))
print("Kolumner:", list(gdf_raw.columns))
gdf_raw.head(3)


Antal objekt: 101
Kolumner: ['Adress', 'Anläggningsnummer', 'Antal_WC', 'Antal_urinoarer', 'Beskrivning', 'Bild', 'CHANGE_DATE', 'CID', 'CREATE_DATE', 'Driftansvar', 'El_Service', 'FEATURE_TYPE_OBJECT_ID', 'FEATURE_TYPE_VERSION_ID', 'Fabrikat', 'Färg', 'Historisk_litterering', 'Huvudman', 'Index', 'Innehåller', 'Installerad', 'Litterering', 'MAIN_ATTRIBUTE_DESCRIPTION', 'MAIN_ATTRIBUTE_VALUE', 'Modell', 'Namn', 'OBJECT_ID', 'Projektnr', 'Pumpgrop', 'Status', 'Tillgänglighetsanpassad', 'VALID_FROM', 'VALID_TO', 'VERSION_ID', 'Vattenservice', 'fid', 'FEATURE_TYPE_NAME', 'geometry']


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: GPKG: unrecognized user_version=0x00000000 (0) on 'data/raw/Toalett_Punkt_Sthlm_20260124.gpkg'
  return ogr_read(


,Adress,Anläggningsnummer,Antal_WC,Antal_urinoarer,Beskrivning,Bild,CHANGE_DATE,CID,CREATE_DATE,Driftansvar,...,Pumpgrop,Status,Tillgänglighetsanpassad,VALID_FROM,VALID_TO,VERSION_ID,Vattenservice,fid,FEATURE_TYPE_NAME,geometry
0,Frödingsvägen 13,G84,NaN,NaN,Inbyggd i byggnad,None,2.025021e+11,1.659929e+15,2.019021e+11,Miljötekniska,...,Nej,I funktion,Ja,2.019021e+11,None,1,None,1,Toalett,POINT (150163.422 6579429.646)
1,Norr Mälarstrand 3 - kajen,G60,NaN,NaN,"Tillgänglighetsanpassad WC, 2 x WC och 1 x uri...",None,2.024012e+11,1.538073e+15,2.018023e+11,JCDecaux,...,Nej,I funktion,Ja,2.018022e+11,None,1,None,2,Toalett,POINT (152988.491 6579085.031)
2,Fjällgatan 37,G85,NaN,NaN,Inbyggd endast i drift sommartid,None,2.022091e+11,1.659930e+15,2.019021e+11,Fjällgatans cafestuga,...,Nej,I funktion,Ja,2.019021e+11,None,1,None,3,Toalett,POINT (154949.341 6578026.085)


In [4]:
from pathlib import Path
def build_history(processed_dir="data/processed"):
    records = []

    for path in sorted(glob.glob(f"{processed_dir}/stockholm_*.parquet")):
        date = Path(path).stem.split("_")[-1]
        df = pd.read_parquet(path)

        for _, r in df.iterrows():
            records.append({
                "stable_id": r.stable_id,
                "date": date,
                "lat": r.lat,
                "lon": r.lon,
                "opening_hours": r.opening_hours,
                "accessibility": r.accessibility,
                "name": r.name,
            })

    return pd.DataFrame(records)

def popup_with_history(row, df_history):
    return folium.Popup(
        f"""
        <b>Namn:</b> {row.name_old}<br>
        <b>Ändring:</b> Attribut ändrade<br><br>
        <b>Historik:</b><br>
        {summarize_history(df_history, row.stable_id)}<br><br>
        <a href="https://www.openstreetmap.org/?mlat={row.lat_new}&mlon={row.lon_new}#map=19/{row.lat_new}/{row.lon_new}" target="_blank">
            Visa i OSM
        </a>
        """,
        max_width=400
    )


In [5]:
df_history = build_history()
print("Historikposter:", len(df_history))


Historikposter: 606


In [6]:
def summarize_history(df_hist, stable_id):
    h = df_hist[df_hist.stable_id == stable_id].sort_values("date")

    if h.empty:
        return "Ingen historik"

    first = h.iloc[0]["date"]
    last = h.iloc[-1]["date"]
    count = len(h)

    moved = (
        h[["lat", "lon"]]
        .diff()
        .abs()
        .sum(axis=1)
        .fillna(0)
        .gt(0.00001)
        .any()
    )

    status_changes = (
        h["opening_hours"].nunique() > 1 or
        h["accessibility"].nunique() > 1
    )

    lines = [
        f"Första observation: {first}",
        f"Senaste observation: {last}",
        f"Antal versioner: {count}",
        f"Flyttad över tid: {'Ja' if moved else 'Nej'}",
        f"Status ändrad: {'Ja' if status_changes else 'Nej'}",
    ]

    return "<br>".join(lines)


In [7]:
import folium 
mdiff = folium.Map(
    location=[59.3293, 18.0686],
    zoom_start=11,
    tiles="OpenStreetMap"
)


In [8]:
def popup(d):
    return folium.Popup(
        "<br>".join(f"<b>{k}</b>: {v}" for k, v in d.items()),
        max_width=350
    )


In [9]:
def timeline_table(df_hist, stable_id, limit=5):
    h = df_hist[df_hist.stable_id == stable_id].sort_values("date")
    if h.empty:
        return "Ingen historik"

    rows = h.tail(limit)

    html = "<table border='1' style='border-collapse:collapse;font-size:12px;'>"
    html += "<tr><th>Datum</th><th>Öppettider</th><th>Tillg.</th></tr>"

    for _, r in rows.iterrows():
        html += (
            f"<tr>"
            f"<td>{r.date}</td>"
            f"<td>{r.opening_hours}</td>"
            f"<td>{r.accessibility}</td>"
            f"</tr>"
        )

    html += "</table>"
    if len(h) > limit:
        html += f"<i>Visar {limit} av {len(h)} versioner</i>"

    return html


In [10]:
from datetime import datetime

def inactive_days(df_hist, stable_id):
    h = df_hist[df_hist.stable_id == stable_id].sort_values("date")
    if len(h) < 2:
        return 0

    last_change = h[h["opening_hours"].ne(h["opening_hours"].shift()) |
                    h["accessibility"].ne(h["accessibility"].shift())]

    if last_change.empty:
        first = datetime.strptime(h.iloc[0]["date"], "%Y%m%d")
    else:
        first = datetime.strptime(last_change.iloc[-1]["date"], "%Y%m%d")

    last = datetime.strptime(h.iloc[-1]["date"], "%Y%m%d")
    return (last - first).days


In [11]:
def stockholm_toalett_adapter(gdf):
    df = gdf.copy()

    # säkerställ punktgeometri
    df = df[df.geometry.notnull()]
    df["lon"] = df.geometry.x
    df["lat"] = df.geometry.y

    out = pd.DataFrame({
        "name": df.get("NAMN", df.get("namn")),
        "lat": df["lat"],
        "lon": df["lon"],
        "opening_hours": df.get("OPPETTIDER", df.get("oppettider")),
        "accessibility": df.get("TILLGANG", df.get("tillganglighet")),
    })

    # rensa minimalt
    out["name"] = out["name"].astype(str).str.strip()

    # stabilt ID (≈ 1 m)
    out["stable_id"] = (
        out["lat"].round(5).astype(str)
        + "_"
        + out["lon"].round(5).astype(str)
    )

    return out.dropna(subset=["lat", "lon"])


In [12]:
df_new = stockholm_toalett_adapter(gdf_raw)
print("Adapter-resultat:", len(df_new))

Adapter-resultat: 101


In [13]:
df_old = None
added = pd.DataFrame()
removed = pd.DataFrame()
changed_geom = pd.DataFrame()
changed_attrs = pd.DataFrame()

if df_old is not None:
    added = df_new[~df_new["stable_id"].isin(df_old["stable_id"])]
    removed = df_old[~df_old["stable_id"].isin(df_new["stable_id"])]

    merged = df_old.merge(
        df_new,
        on="stable_id",
        suffixes=("_old", "_new")
    )

    changed_geom = merged[
        (merged["lat_old"] != merged["lat_new"]) |
        (merged["lon_old"] != merged["lon_new"])
    ]

    changed_attrs = merged[
        (merged["opening_hours_old"] != merged["opening_hours_new"]) |
        (merged["accessibility_old"] != merged["accessibility_new"])
    ]


In [14]:
import folium 
fg_added = folium.FeatureGroup(name="🟢 Tillkomna")
today = date.today().strftime("%Y%m%d") 



for _, r in added.iterrows():
    popup_html = f"""
        <b>Namn:</b> {r.name_old}<br>
        <b>Ändring:</b> Attribut ändrade<br><br>
        
        <b>Historik:</b><br>
        {timeline_table(df_history, r.stable_id)}<br><br>
        
        <a href="https://www.openstreetmap.org/?mlat={r.lat_new}&mlon={r.lon_new}#map=19/{r.lat_new}/{r.lon_new}" target="_blank">
        Visa i OSM
        </a>
        """
    folium.CircleMarker(
        [r.lat, r.lon],
        radius=6,
        color="green",
        fill=True,
        fill_opacity=0.8,
        popup=popup({
            "Namn": r.name,
            "Ändring": "Tillkommen"
        })
    ).add_to(fg_added)

fg_added.add_to(mdiff) 

fg_removed = folium.FeatureGroup(name="🔴 Borttagna")

for _, r in removed.iterrows():
    folium.CircleMarker(
        [r.lat, r.lon],
        radius=6,
        color="red",
        fill=True,
        fill_opacity=0.8,
        popup=popup({
            "Namn": r.name,
            "Ändring": "Borttagen"
        })
    ).add_to(fg_removed)

fg_removed.add_to(mdiff)

fg_moved = folium.FeatureGroup(name="🟠 Flyttade")

for _, r in changed_geom.iterrows():
    old = (r.lat_old, r.lon_old)
    new = (r.lat_new, r.lon_new)

    folium.PolyLine([old, new], color="orange", weight=2).add_to(fg_moved)

    folium.CircleMarker(old, radius=5, color="orange", fill=True).add_to(fg_moved)
    folium.CircleMarker(
        new, radius=7, color="orange", fill=True,
        popup=popup({
            "Namn": r.name_old,
            "Gammal position": old,
            "Ny position": new
        })
    ).add_to(fg_moved)

fg_moved.add_to(mdiff)

fg_changed = folium.FeatureGroup(name="🟡 Ändrade attribut")

for _, r in changed_attrs.iterrows():
    folium.CircleMarker(
        [r.lat_new, r.lon_new],
        radius=6,
        color="gold",
        fill=True,
        fill_opacity=0.8,
        popup=popup_with_history(r, df_history)
    ).add_to(fg_changed)


fg_changed.add_to(mdiff)

folium.LayerControl(collapsed=False).add_to(mdiff)

legend_html = f"""
<div style="
 position: fixed;
 bottom: 30px;
 left: 30px;
 z-index: 9999;
 background-color: white;
 padding: 10px;
 border: 2px solid grey;
 font-size: 14px;
">
<b>Legend – Toaletter Stockholm (diff)</b><br>
<span style="color:green;">●</span> Tillkommen<br>
<span style="color:red;">●</span> Borttagen<br>
<span style="color:orange;">●</span> Flyttad<br>
<span style="color:gold;">●</span> Attribut ändrat<br>
<hr> Analys: versionsjämförelse {today}<br /><br />
<b> Mer info</b><br>
• <a href="https://github.com/salgo60/Offentliga-toaletter/issues/8" target="_blank">GitHub Issue #8</a><br>
• <a href="https://dataportalen.stockholm.se/dataportalen/GetMetaDataById?id=LvFeature15303196&showmetadataview" target="_blank">
   Stockholms stads öppna data</a>
</div>
"""
mdiff.get_root().html.add_child(folium.Element(legend_html))

mdiff_Filename = f"Toaletter_sthlm_diff_{today}.html"
mdiff.save(mdiff_Filename)

print("Klar: ", mdiff_Filename)


Klar:  Toaletter_sthlm_diff_20260130.html


In [15]:
mdiff

In [16]:
from pathlib import Path 
Path("data/processed").mkdir(parents=True, exist_ok=True)

processed_files = sorted(glob.glob("data/processed/stockholm_*.parquet"))
df_old = pd.read_parquet(processed_files[-1]) if processed_files else None

df_new.to_parquet(f"data/processed/stockholm_{today}.parquet")

added = removed = changed_geom = changed_attrs = pd.DataFrame()

if df_old is not None:
    added = df_new[~df_new.stable_id.isin(df_old.stable_id)]
    removed = df_old[~df_old.stable_id.isin(df_new.stable_id)]

    merged = df_old.merge(
        df_new,
        on="stable_id",
        suffixes=("_old", "_new")
    )

    changed_geom = merged[
        (merged.lat_old != merged.lat_new) |
        (merged.lon_old != merged.lon_new)
    ]

    changed_attrs = merged[
        (merged.opening_hours_old != merged.opening_hours_new) |
        (merged.accessibility_old != merged.accessibility_new)
    ]

print(
    "Tillkomna:", len(added),
    "Borttagna:", len(removed),
    "Flyttade:", len(changed_geom),
    "Ändrade attribut:", len(changed_attrs)
)


Tillkomna: 0 Borttagna: 0 Flyttade: 0 Ändrade attribut: 101


## Karta Offentliga toaletter 
* visat karta med OSM offentliga toaletter och Stockholms stad

In [17]:
import geopandas as gpd

# Läs in geopackage-filen
gdf = gpd.read_file(SthlmfileName)

# Visa grundläggande information
print(gdf.head())
print(gdf.crs)
print(gdf.columns)

                           Adress Anläggningsnummer  Antal_WC  \
0                Frödingsvägen 13               G84       NaN   
1      Norr Mälarstrand 3 - kajen               G60       NaN   
2                   Fjällgatan 37               G85       NaN   
3                   Sveavägen 39A               G67       NaN   
4  Norra Bantorget - Vasagatan 59               G14       1.0   

   Antal_urinoarer                                        Beskrivning  Bild  \
0              NaN                                  Inbyggd i byggnad  None   
1              NaN  Tillgänglighetsanpassad WC, 2 x WC och 1 x uri...  None   
2              NaN                   Inbyggd endast i drift sommartid  None   
3              NaN                            Tillgänglighetsanpassad  None   
4              NaN                         Tillgänglighetsanpassad WC  None   

    CHANGE_DATE           CID   CREATE_DATE            Driftansvar  ...  \
0  2.025021e+11  1.659929e+15  2.019021e+11          Miljöt

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: GPKG: unrecognized user_version=0x00000000 (0) on 'data/raw/Toalett_Punkt_Sthlm_20260123.gpkg'
  return ogr_read(


In [18]:
def convert_misread_date(x):
    try:
        if pd.isna(x):
            return None
        # gör till int, ta bara siffrorna
        s = str(int(float(x)))
        # om 8 tecken → YYYYMMDD
        if len(s) == 8:
            return pd.to_datetime(s, format="%Y%m%d", errors="coerce")
        return None
    except Exception:
        return None

for col in [c for c in gdf.columns if "DATE" in c.upper()]:
    gdf[col + "_fmt"] = gdf[col].apply(convert_misread_date)

In [19]:
import geopandas as gpd
import pandas as pd

# Läs in
gdf = gpd.read_file(SthlmfileName)

# Beräkna centroid i meter-CRS (SWEREF99 TM) och konvertera till WGS84
gdf_proj = gdf.to_crs(epsg=3006)
gdf["geometry_centroid"] = gdf_proj.geometry.centroid.to_crs(epsg=4326)

# Lägg till lat/lon
gdf["lon"] = gdf["geometry_centroid"].x
gdf["lat"] = gdf["geometry_centroid"].y

# Lista möjliga datumkolumner
date_cols = [c for c in gdf.columns if "DATE" in c.upper() or "DATUM" in c.upper()]
print("🕒 Datumkolumner:", date_cols)

# Funktion för att försöka tolka stora siffror som tidsstämplar
def convert_date(x):
    try:
        # Om NaN
        if pd.isna(x):
            return None
        # Om redan datetime
        if isinstance(x, pd.Timestamp):
            return x
        # Om det ser ut som 1.738443e+15 (millisekunder)
        x = float(x)
        if x > 1e12:  # troligen millisekunder
            return pd.to_datetime(x, unit="ms")
        elif x > 1e9:  # sekunder
            return pd.to_datetime(x, unit="s")
        else:
            return None
    except Exception:
        return None

# Konvertera alla identifierade datumfält
for col in date_cols:
    gdf[col + "_fmt"] = gdf[col].apply(convert_date)

# Visa exempel på konverterade datum
print(gdf[[c for c in gdf.columns if "DATE" in c or "fmt" in c]].head())

gdf.drop(columns=["geometry", "geometry_centroid"]).to_csv("SthlmfileName_Toalett_fix.csv", index=False)
print("✅ Sparade 'SthlmfileName_Toalett_fix.csv' med riktiga datum och koordinater")

🕒 Datumkolumner: ['CHANGE_DATE', 'CREATE_DATE']
    CHANGE_DATE   CREATE_DATE CHANGE_DATE_fmt CREATE_DATE_fmt
0  2.025021e+11  2.019021e+11            None            None
1  2.024012e+11  2.018023e+11            None            None
2  2.022091e+11  2.019021e+11            None            None
3  2.024012e+11  2.018023e+11            None            None
4  2.024012e+11  2.018011e+11            None            None
✅ Sparade 'SthlmfileName_Toalett_fix.csv' med riktiga datum och koordinater


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyogrio/raw.py:198: RuntimeWarning: GPKG: unrecognized user_version=0x00000000 (0) on 'data/raw/Toalett_Punkt_Sthlm_20260123.gpkg'
  return ogr_read(


In [20]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 42 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   Adress                      101 non-null    object  
 1   Anläggningsnummer           101 non-null    object  
 2   Antal_WC                    4 non-null      float64 
 3   Antal_urinoarer             2 non-null      float64 
 4   Beskrivning                 75 non-null     object  
 5   Bild                        1 non-null      object  
 6   CHANGE_DATE                 101 non-null    float64 
 7   CID                         101 non-null    float64 
 8   CREATE_DATE                 101 non-null    float64 
 9   Driftansvar                 101 non-null    object  
 10  El_Service                  0 non-null      object  
 11  FEATURE_TYPE_OBJECT_ID      101 non-null    int64   
 12  FEATURE_TYPE_VERSION_ID     101 non-null    int16   
 13  Fabrikat    

In [21]:
gdf.head()

,Adress,Anläggningsnummer,Antal_WC,Antal_urinoarer,Beskrivning,Bild,CHANGE_DATE,CID,CREATE_DATE,Driftansvar,...,VERSION_ID,Vattenservice,fid,FEATURE_TYPE_NAME,geometry,geometry_centroid,lon,lat,CHANGE_DATE_fmt,CREATE_DATE_fmt
0,Frödingsvägen 13,G84,NaN,NaN,Inbyggd i byggnad,None,2.025021e+11,1.659929e+15,2.019021e+11,Miljötekniska,...,1,None,1,Toalett,POINT (150163.422 6579429.646),POINT (18.00287 59.32999),18.002871,59.329993,None,None
1,Norr Mälarstrand 3 - kajen,G60,NaN,NaN,"Tillgänglighetsanpassad WC, 2 x WC och 1 x uri...",None,2.024012e+11,1.538073e+15,2.018023e+11,JCDecaux,...,1,None,2,Toalett,POINT (152988.491 6579085.031),POINT (18.05249 59.32689),18.052495,59.326889,None,None
2,Fjällgatan 37,G85,NaN,NaN,Inbyggd endast i drift sommartid,None,2.022091e+11,1.659930e+15,2.019021e+11,Fjällgatans cafestuga,...,1,None,3,Toalett,POINT (154949.341 6578026.085),POINT (18.08691 59.31736),18.086914,59.317365,None,None
3,Sveavägen 39A,G67,NaN,NaN,Tillgänglighetsanpassad,None,2.024012e+11,1.538074e+15,2.018023e+11,JCDecaux,...,1,None,4,Toalett,POINT (153474.156 6580313.629),POINT (18.06105 59.33791),18.061045,59.337914,None,None
4,Norra Bantorget - Vasagatan 59,G14,1.0,NaN,Tillgänglighetsanpassad WC,None,2.024012e+11,1.530323e+15,2.018011e+11,JCDecaux,...,1,None,5,Toalett,POINT (153148.155 6579983.768),POINT (18.05531 59.33496),18.055312,59.334955,None,None


In [22]:
gdf.describe()

,Antal_WC,Antal_urinoarer,CHANGE_DATE,CID,CREATE_DATE,FEATURE_TYPE_OBJECT_ID,FEATURE_TYPE_VERSION_ID,OBJECT_ID,VALID_FROM,VERSION_ID,fid,lon,lat
count,4.0,2.0,1.010000e+02,1.010000e+02,1.010000e+02,101.0,101.0,1.010000e+02,1.010000e+02,101.0,101.000000,101.000000,101.000000
mean,1.0,1.0,2.024175e+11,1.747852e+15,2.019862e+11,15303196.0,1.0,1.747852e+07,2.019861e+11,1.0,51.000000,18.028833,59.323680
std,0.0,0.0,7.942318e+07,3.398323e+14,2.930167e+08,0.0,0.0,3.398322e+06,2.930255e+08,0.0,29.300171,0.064846,0.032642
min,1.0,1.0,2.022090e+11,1.530321e+15,2.018011e+11,15303196.0,1.0,1.530321e+07,2.018011e+11,1.0,1.000000,17.843198,59.243971
25%,1.0,1.0,2.024012e+11,1.530326e+15,2.018011e+11,15303196.0,1.0,1.530326e+07,2.018011e+11,1.0,26.000000,18.002871,59.301593
50%,1.0,1.0,2.024012e+11,1.538074e+15,2.018023e+11,15303196.0,1.0,1.538074e+07,2.018022e+11,1.0,51.000000,18.052149,59.326010
75%,1.0,1.0,2.025101e+11,1.775757e+15,2.020092e+11,15303196.0,1.0,1.775757e+07,2.020092e+11,1.0,76.000000,18.075276,59.339646
max,1.0,1.0,2.025102e+11,2.358270e+15,2.025101e+11,15303196.0,1.0,2.358270e+07,2.025101e+11,1.0,101.000000,18.130955,59.413180


In [23]:
gdf["Antal_WC"].value_counts() 

Antal_WC
1.0    4
Name: count, dtype: int64

In [24]:
gdf["Antal_urinoarer"].value_counts() 

Antal_urinoarer
1.0    2
Name: count, dtype: int64

In [25]:
gdf["Beskrivning"].value_counts()

Beskrivning
Tillgänglighetsanpassad                                         26
Tillgänglighetsanpassad WC                                      25
Tillgänglighetsanpassad, integrerad cykelpump                    7
Tillgänglighetsanpassad WC, 2 x WC, 1 x urinoar                  6
Tillgänglighets anpassad                                         2
Tillgänglighetsanpassad, integrerad cykelpump, kortbetalning     2
Inbyggd i byggnad                                                1
Tillgänglighetsanpassad WC, 2 x WC och 1 x urionar               1
Inbyggd endast i drift sommartid                                 1
3st toaletter 1st tillgänglighetsanpassad (säsongs öppna)        1
Tillgänglighetsanpassad, kortbetalning                           1
Urinoarer                                                        1
Tillgänglighets anspassad                                        1
Name: count, dtype: int64

### Check if Stockholms stads data has been updated

In [26]:
#gdf["CREATE_DATE"].value_counts() 
gdf["CREATE_DATE"].astype("Int64").astype(str).str.len().value_counts()


CREATE_DATE
12    101
Name: count, dtype: int64

In [27]:
gdf["CREATE_DATE_DT"] = pd.to_datetime(
    gdf["CREATE_DATE"].astype("Int64").astype(str),
    format="%Y%m%d%H%M",
    errors="coerce"
)

In [28]:
gdf["CHANGE_DATE_DT"] = pd.to_datetime(
    gdf["CHANGE_DATE"].astype("Int64").astype(str),
    format="%Y%m%d%H%M",
    errors="coerce"
)

In [29]:
gdf[["CREATE_DATE", "CREATE_DATE_DT"]].sample(10)

,CREATE_DATE,CREATE_DATE_DT
100,2.018011e+11,2018-01-08 16:57:00
35,2.025101e+11,2025-10-10 14:42:00
66,2.020023e+11,2020-02-25 15:34:00
93,2.018023e+11,2018-02-28 21:13:00
49,2.018023e+11,2018-02-28 21:13:00
16,2.018011e+11,2018-01-08 16:57:00
13,2.025101e+11,2025-10-10 10:20:00
32,2.018023e+11,2018-02-28 21:13:00
72,2.025101e+11,2025-10-10 12:29:00
8,2.018011e+11,2018-01-08 16:57:00


In [30]:
gdf_sorted_create = gdf.sort_values(by="CREATE_DATE_DT", ascending=False) 
gdf_sorted_create[["CREATE_DATE", "CREATE_DATE_DT"]]

,CREATE_DATE,CREATE_DATE_DT
84,2.025101e+11,2025-10-13 07:35:00
83,2.025101e+11,2025-10-10 15:07:00
81,2.025101e+11,2025-10-10 15:02:00
92,2.025101e+11,2025-10-10 14:58:00
87,2.025101e+11,2025-10-10 14:54:00
...,...,...
61,2.018011e+11,2018-01-08 16:57:00
62,2.018011e+11,2018-01-08 16:57:00
63,2.018011e+11,2018-01-08 16:57:00
65,2.018011e+11,2018-01-08 16:57:00


In [31]:
gdf_sorted_change = gdf.sort_values(by="CHANGE_DATE_DT", ascending=False) 
gdf_sorted_change[["CHANGE_DATE", "CHANGE_DATE_DT"]]

,CHANGE_DATE,CHANGE_DATE_DT
85,2.025102e+11,2025-10-20 06:12:00
17,2.025101e+11,2025-10-13 09:01:00
15,2.025101e+11,2025-10-13 07:51:00
51,2.025101e+11,2025-10-13 07:46:00
84,2.025101e+11,2025-10-13 07:35:00
...,...,...
36,2.022090e+11,2022-09-01 13:16:00
30,2.022090e+11,2022-09-01 13:08:00
25,2.022090e+11,2022-09-01 12:59:00
42,2.022090e+11,2022-09-01 07:58:00


In [32]:
gdf_sorted_change["CHANGE_DATE_DAY"] = gdf_sorted_change["CHANGE_DATE_DT"].dt.date

gdf_sorted_change.groupby("CHANGE_DATE_DAY").size().reset_index(name="antal")

,CHANGE_DATE_DAY,antal
0,2022-09-01,6
1,2022-09-06,1
2,2022-10-05,1
3,2023-10-30,1
4,2024-01-17,56
5,2024-01-23,3
6,2024-02-05,1
7,2024-06-04,1
8,2024-10-10,1
9,2025-02-11,1


In [33]:
gdf[["CHANGE_DATE", "CHANGE_DATE_DT"]].sample(10)

,CHANGE_DATE,CHANGE_DATE_DT
99,2.024012e+11,2024-01-17 09:13:00
34,2.025101e+11,2025-10-10 14:46:00
92,2.025101e+11,2025-10-10 15:00:00
37,2.022090e+11,2022-09-01 13:17:00
22,2.024012e+11,2024-01-17 08:52:00
24,2.025101e+11,2025-10-10 09:33:00
96,2.025101e+11,2025-10-10 11:29:00
67,2.024012e+11,2024-01-17 09:06:00
74,2.025101e+11,2025-10-10 11:28:00
32,2.023103e+11,2023-10-30 06:45:00


In [34]:
gdf["Driftansvar"].value_counts()

Driftansvar
JCDecaux                 47
Danfo                    31
Miljötekniska            22
Fjällgatans cafestuga     1
Name: count, dtype: int64

In [35]:
from shapely.geometry import base

for col in gdf.columns:
    has_geom = gdf[col].apply(lambda x: isinstance(x, base.BaseGeometry)).any()
    if has_geom:
        print(f"🧱 Kolumn '{col}' innehåller geometrier")


🧱 Kolumn 'geometry' innehåller geometrier
🧱 Kolumn 'geometry_centroid' innehåller geometrier


In [36]:
from urllib.parse import unquote

def build_commons_thumbnail(tags: dict, width: int = 240) -> str:
    image_url = tags.get("image")
    if not image_url:
        return ""

    if "/wiki/File:" not in image_url:
        return ""

    filename = image_url.split("/wiki/File:")[-1]
    filename = unquote(filename)  # hantera %C3%A5 etc korrekt

    thumb_url = (
        "https://commons.wikimedia.org/wiki/Special:FilePath/"
        f"{filename}?width={width}"
    )

    return f"""
    <div style="margin-top:6px;text-align:center;">
      <a href="{image_url}" target="_blank">
        <img src="{thumb_url}"
             style="max-width:100%;
                    border-radius:6px;
                    box-shadow:0 1px 4px rgba(0,0,0,0.3);"
             loading="lazy"
             alt="Wikimedia Commons image">
      </a>
      <div style="font-size:12px;color:#555;margin-top:2px;">
        Klicka för full storlek (Wikimedia Commons)
      </div>
    </div>
    """


In [37]:
def build_toilet_tooltip(row):
    operator = row.get("Driftansvar", "")
    description = row.get("Beskrivning", "")
    change_dt = row.get("CHANGE_DATE_DT", "")
    create_dt = row.get("CREATE_DATE_DT", "")

    avtal_html = ""
    if operator == "JCDecaux":
        avtal_html = "<b style='color:#7b2cff;'>⚠ Avtal uppsagt (JCDecaux)</b><br>"

    tooltip_html = f"""
    <div style="font-size:12px;">
      {avtal_html}
      <b>Beskrivning:</b> {description}<br>
      <b>Skapad:</b> {create_dt}<br>
      <b>Ändrad:</b> {change_dt}
    </div>
    """

    return tooltip_html


In [38]:
from urllib.parse import quote

def build_toilet_popup(row, *, include_source=True):    
    lat = row["lat"]
    lon = row["lon"]

    name = "Offentlig toalett"
    address = row.get("Adress", "")
    status = row.get("Status", "")
    operator = row.get("Driftansvar", "Stockholms stad")
    accessible = row.get("Tillgänglighetsanpassad", "")
    facility_id = row.get("Anläggningsnummer", "")
    operator = row.get("Driftansvar", "")
    description = row.get("Beskrivning", "")
    change_dt = row.get("CHANGE_DATE_DT", "")
    create_dt = row.get("CREATE_DATE_DT", "")

    avtal_html = ""
    if operator == "JCDecaux":
        avtal_html = "<b style='color:#7b2cff;'>⚠ Avtal uppsagt (JCDecaux)</b><br>"


    # --- Tillgänglighet ---
    wheelchair_html = {
        "Ja": "♿ <b>Tillgänglig:</b> Ja ✅<br>",
        "Nej": "♿ <b>Tillgänglig:</b> Nej ❌<br>",
    }.get(accessible, "♿ <b>Tillgänglig:</b> Okänd ❔<br>")

    # --- OSM Note ---
    note_text = (
        f"Kommunal toalett enligt Stockholms stad. "
        f"Adress: {address}. "
        f"Tillgänglig: {accessible}. "
        f"Anläggningsnummer: {facility_id}."
    )
    note_url = (
        f"https://www.openstreetmap.org/note/new"
        f"#map=18/{lat}/{lon}&text={quote(note_text)}"
    )

    mapcomplete = f"https://mapcomplete.org/toilets.html?z=17&lat={lat}&lon={lon}#about_theme"
    wheelmap = f"https://wheelmap.org/map#/?lat={lat}&lon={lon}&zoom=17"
    # --- App / editor-länkar ---
    google_maps = f"https://maps.google.com/?q={lat},{lon}"
    streetview = f"https://www.google.com/maps/@?api=1&map_action=pano&viewpoint={lat},{lon}"
    wikishootme = f"https://wikishootme.toolforge.org/#lat={lat}&lng={lon}&zoom=17"
    
    menu_html = f"""
        <div style='margin-top:8px; text-align:center;'>
          <a href='{mapcomplete}' target='_blank'
             style='background:#34a853;color:white;padding:6px 10px;
                    border-radius:6px;text-decoration:none;margin:2px;
                    display:inline-block;font-size:13px;'>
             🚻 MapComplete – Toilets
          </a>
        
          <a href='{wheelmap}' target='_blank'
             style='background:#7c3aed;color:white;padding:6px 10px;
                    border-radius:6px;text-decoration:none;margin:2px;
                    display:inline-block;font-size:13px;'>
             ♿ Wheelmap
          </a>
        
          <a href='{note_url}' target='_blank'
             style='background:#fbbc04;color:black;padding:6px 10px;
                    border-radius:6px;text-decoration:none;margin:2px;
                    display:inline-block;font-size:13px;'>
             📝 OSM Note
          </a>
        </div>
        """

    tools_html = f"""
    <details style='margin-top:6px; text-align:center;'>
      <summary style='cursor:pointer;font-weight:bold;'>🔗 Fler kartverktyg</summary>
      <a href='https://www.openstreetmap.org/edit#map=18/{lat}/{lon}' target='_blank'>✏️ iD Editor</a><br>
      <a href='https://www.mapillary.com/app/?lat={lat}&lng={lon}&z=17' target='_blank'>🟢 Mapillary</a><br>
      <a href='{google_maps}' target='_blank'>🗺️ Google Maps</a><br>
      <a href='{streetview}' target='_blank'>👁️ Google Street View</a><br>
      <a href='{wikishootme}' target='_blank'>🧭 Wikishootme</a><br>
    </details>
    """

    source_html = ""
    if include_source:
        source_html ="""<hr style="margin:6px 0;">
          <small>
            <b>Källa:</b>
            <a href="https://dataportalen.stockholm.se/dataportalen/GetMetaDataById?id=LvFeature15303196&showmetadataview"
               target="_blank">
               Stockholms stads öppna data
            </a>
          </small>"""
 
    popup_html = f"""
    <div style='font-family:Arial; font-size:13px; max-width:320px;'>
      <b>🚻 {name}</b><br>
      <b>Adress:</b> {address}<br>
      <b>Status:</b> {status}<br> 
      {avtal_html}
      <b>Beskrivning:</b> {description}<br>
      <b>Skapad:</b> {create_dt}<br>
      <b>Ändrad:</b> {change_dt}
      
      <b>Driftansvar:</b> {operator}<br>
      <b>Anläggningsnr:</b> {facility_id}<br>
      {wheelchair_html}
      📍 <a href='https://www.openstreetmap.org/?mlat={lat}&mlon={lon}#map=18/{lat}/{lon}'
            target='_blank'>{lat:.5f}, {lon:.5f}</a><br>

      {menu_html}
      {tools_html}
      {source_html}
      <!-- OSM_SECTION -->
    </div>
    """
    return popup_html


In [39]:
def build_external_media_links(tags: dict) -> str:
    links = []

    # Wikimedia Commons – image (direkt fil)
    image = tags.get("image")
    if image and image.startswith("https://commons.wikimedia.org/wiki/"):
        links.append(
            f'🖼️ <a href="{image}" target="_blank">Bild (Wikimedia Commons)</a>'
        )

    # Wikimedia Commons – category eller file via wikimedia_commons
    commons = tags.get("wikimedia_commons")
    if commons:
        commons_url = f"https://commons.wikimedia.org/wiki/{commons}"
        links.append(
            f'📂 <a href="{commons_url}" target="_blank">Commons-kategori</a>'
        )

    # Mapillary
    mapillary_key = tags.get("mapillary")
    if mapillary_key and mapillary_key.isdigit():
        mapillary_url = f"https://www.mapillary.com/app/?pKey={mapillary_key}"
        links.append(
            f'📷 <a href="{mapillary_url}" target="_blank">Mapillary-bild</a>'
        )

    # Panoramax – både panoramax och panoramax:*
    panoramax_ids = []

    if "panoramax" in tags:
        panoramax_ids.append(tags["panoramax"])

    for k, v in tags.items():
        if k.startswith("panoramax:") and v:
            panoramax_ids.append(v)

    for pano_id in panoramax_ids:
        pano_url = f"https://api.panoramax.xyz/#focus=pic&pic={pano_id}"
        links.append(
            f'🌐 <a href="{pano_url}" target="_blank">Panoramax-bild</a>'
        )

    if not links:
        return ""

    return (
        "<div style='margin-top:6px;'>"
        "<b>🔗 Media</b><br>"
        + "<br>".join(links) +
        "</div>"
    )


In [40]:
def build_osm_toilet_popup(
    osm_el: dict,
    *,
    lat: float,
    lon: float,
    lifecycle: str
) -> str:

    tags = osm_el.get("tags", {})
    osm_id = osm_el["id"]
    osm_type = osm_el["type"]

    row = {
        "lat": lat,
        "lon": lon,
        "Tillgänglighetsanpassad": tags.get("wheelchair", ""),
    }

    base_popup = build_toilet_popup(row, include_source=False)

    osm_object_url = f"https://www.openstreetmap.org/{osm_type}/{osm_id}"
    tags_html = render_osm_tags(tags)
    warnings_html = get_data_quality_warnings(tags)
    mapcomplete_link = build_mapcomplete_link(lat, lon, tags)

    media_links_html = build_external_media_links(tags)
    commons_thumb_html = build_commons_thumbnail(tags)
    if lifecycle == "disused":
        status_html = "🚫 Ur bruk (disused)"
    elif lifecycle == "active":
        status_html = "✅ I drift"
    else:
        status_html = "❓ Okänd status"

    status_block = f"""
    <b>Status:</b> {status_html}<br>
    """
    status_section = f"""
    <div style="margin-bottom:6px;">
      <b>🛠 Status & underhåll</b><br>
      Status: {status_html}<br>
    """

    if lifecycle == "disused":
        reopening = tags.get("expected_opening", "")
        note = tags.get("note", "")
        check_date = tags.get("check_date", "")
    
        if check_date:
            status_section += f"Senast kontrollerad: {check_date}<br>"
        if reopening:
            status_section += f"Planerad öppning: {reopening}<br>"
        if note:
            status_section += f"Kommentar: {note}<br>"
    
        status_section += "</div><hr style='margin:6px 0;'>"

    base_popup = status_section + base_popup

    osm_section = f"""
    <hr style="margin:6px 0;">
    <b>🧾 OpenStreetMap</b><br>
    <a href="{osm_object_url}" target="_blank">
      {osm_type.capitalize()} {osm_id}
    </a><br>
    {commons_thumb_html}
    {media_links_html}

    {warnings_html}

    <details style="margin-top:6px;">
      <summary style="cursor:pointer;font-weight:bold;">
        🏷️ OSM Tags
      </summary>
      {tags_html}
    </details>
    """
    return base_popup.replace("<!-- OSM_SECTION -->", osm_section)    


In [41]:
def build_mapcomplete_link(lat, lon, tags: dict) -> str:
    """
    Build MapComplete Toilets link with prefilled tags.
    """
    base = "https://mapcomplete.org/toilets.html"

    allowed_keys = {
        "wheelchair",
        "fee",
        "access",
        "unisex",
        "changing_table",
        "toilets:disposal",
        "toilets:position",
        "drinking_water",
    }

    preset_tags = {
        k: v for k, v in tags.items()
        if k in allowed_keys
    }

    tag_fragment = "&".join(
        f"{quote(k)}={quote(v)}" for k, v in preset_tags.items()
    )

    return (
        f"{base}?z=18&lat={lat}&lon={lon}"
        f"#preset=toilets"
        + (f"&{tag_fragment}" if tag_fragment else "")
    )


In [42]:
def get_data_quality_warnings(tags: dict, stale_years: int = 3) -> str:
    """
    Highlight questionable OSM data such as fixme or old check_date.
    """
    warnings = []

    # fixme=* is always suspicious
    if "fixme" in tags:
        warnings.append(
            f"⚠️ <b>fixme:</b> {tags.get('fixme')}"
        )

    # check_date staleness
    cd = tags.get("check_date")
    parsed = parse_osm_date(cd) if cd else None
    if parsed:
        age = date.today().year - parsed.year
        if age >= stale_years:
            warnings.append(
                f"⏰ <b>check_date:</b> {cd} (≈ {age} years old)"
            )

    if not warnings:
        return ""

    return (
        "<div style='background:#fff3cd;"
        "border:1px solid #ffecb5;"
        "padding:6px;border-radius:6px;"
        "margin-top:6px;font-size:12px;'>"
        "<b>Datakvalitet</b><br>"
        + "<br>".join(warnings) +
        "</div>"
    )


In [43]:
def render_osm_tags(tags: dict) -> str:
    """
    Render OSM tags as HTML with links to OSM Wiki for each key.
    """
    if not tags:
        return "<i>No tags</i>"

    rows = []
    for k, v in sorted(tags.items()):
        wiki = f"https://wiki.openstreetmap.org/wiki/Key:{quote(k)}"
        rows.append(
            f"<tr>"
            f"<td style='padding-right:6px;'><a href='{wiki}' target='_blank'><code>{k}</code></a></td>"
            f"<td><code>{v}</code></td>"
            f"</tr>"
        )

    return (
        "<table style='border-collapse:collapse;font-size:12px;'>"
        + "".join(rows)
        + "</table>"
    )


In [44]:
from datetime import date, datetime
def parse_osm_date(value: str):
    """
    Parse OSM-style date (YYYY-MM-DD). Return date or None.
    """
    try:
        return datetime.strptime(value, "%Y-%m-%d").date()
    except Exception:
        return None


### Open Street Map layer

In [45]:
import requests
import json
import os
import time
from pathlib import Path  

def get_lat_lon(el):
    if el["type"] == "node":
        return el.get("lat"), el.get("lon")
    if el["type"] in ("way", "relation"):
        center = el.get("center")
        if center:
            return center.get("lat"), center.get("lon")
    return None, None

# 🗺️ Overpass API
OVERPASS_URL = "https://overpass-api.de/api/interpreter"

# 🔍 Overpass query toilets Stockholms stad
OVERPASS_QUERY = """
[out:json][timeout:60];
area(3600398021)->.searchArea;
nwr["amenity"="toilets"](area.searchArea);
out center;
"""
# 🔍 Overpass query toilets Stockholms stad and "disused:amenity": "toilets"
OVERPASS_QUERY = """ 
[out:json][timeout:60];
area(3600398021)->.searchArea;
(
  nwr["amenity"="toilets"](area.searchArea);
  nwr["disused:amenity"="toilets"](area.searchArea);
);
out center;
"""


# 💾 Cache settings
CACHE_FILE = Path("osm_Sthlmkommun_cache.json")
#CACHE_TTL = 60  # seconds (dev default)
CACHE_TTL = 60 * 60 * 24 * 7  # 7 days (prod)

def is_cache_valid(cache_file: Path, ttl: int) -> bool:
    if not cache_file.exists():
        return False
    age = time.time() - cache_file.stat().st_mtime
    return age < ttl

def load_cache(cache_file: Path):
    with cache_file.open("r", encoding="utf-8") as f:
        return json.load(f)

def save_cache(cache_file: Path, data: dict):
    with cache_file.open("w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def fetch_overpass(query: str) -> dict:
    response = requests.get(
        OVERPASS_URL,
        params={"data": query},
        timeout=90,
    )
    response.raise_for_status()
    return response.json()

def get_osm_toilets(
    *,
    use_cache: bool = True,
    force_refresh: bool = False,
) -> dict:
    """
    use_cache=True        → allow cache usage
    force_refresh=True    → bypass cache even if valid
    """

    if use_cache and not force_refresh and is_cache_valid(CACHE_FILE, CACHE_TTL):
        age = int(time.time() - CACHE_FILE.stat().st_mtime)
        print(f"📦 Using cache ({age}s old)")
        return load_cache(CACHE_FILE)

    print("🌍 Fetching toilets from Overpass API…")

    try:
        data = fetch_overpass(OVERPASS_QUERY)
        save_cache(CACHE_FILE, data)
        print(f"✅ Cached {len(data.get('elements', []))} objects")
        return data

    except Exception as e:
        print(f"❌ Overpass error: {e}")

        if CACHE_FILE.exists():
            print("⚠️ Falling back to stale cache")
            return load_cache(CACHE_FILE)

        raise  # no cache to fall back on

# 🚀 Example usage

# Normal dev run (cache enabled)
#osm_data = get_osm_toilets()

# Force refresh (bypass cache)  
osm_data = get_osm_toilets(force_refresh=True)

print(f"🐾 Found {len(osm_data.get('elements', []))} toilets")


🌍 Fetching toilets from Overpass API…
✅ Cached 254 objects
🐾 Found 254 toilets


In [46]:
osm_data

{'version': 0.6,
 'generator': 'Overpass API 0.7.62.10 2d4cfc48',
 'osm3s': {'timestamp_osm_base': '2026-01-30T01:02:35Z',
  'timestamp_areas_base': '2026-01-23T01:22:35Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 29835248,
   'lat': 59.3385713,
   'lon': 18.0553167,
   'tags': {'changing_table': 'yes',
    'check_date': '2026-01-27',
    'disused:amenity': 'toilets',
    'fee': 'no',
    'fixme': 'Stängd toalett. Nya offentliga toaletter kommer att placeras ut med start i mars 2026.',
    'image': 'https://commons.wikimedia.org/wiki/File:Stockholms_stads_offentliga_toalett_Tegnerlunden_2026_jan_01.jpg',
    'image:license': 'CC0',
    'mapillary': '3720571278251388',
    'name': 'Toialett Tegnerlunden',
    'note': 'Closed due to Stockholm City supplier change; planned new toilets later in year',
    'operator': 'Stockholms stad',
    'source:access': 'https://tra

In [47]:
with open("osm_data.json", "w", encoding="utf-8") as f:
    json.dump(osm_data, f, indent=2, ensure_ascii=False)

# Generate Map

In [48]:
import folium 
m = folium.Map(location=[59.3293, 18.0686], zoom_start=11)

gdf.to_csv("Sthlm.csv")

In [49]:
from shapely.geometry import base

# Ta bort alla geometrier utom den huvudsakliga
gdf_clean = gdf.copy()

# ta bort "geometry_centroid" och andra geometrikolumner
geom_cols = [c for c in gdf_clean.columns 
             if c != "geometry" and gdf_clean[c].apply(lambda x: isinstance(x, base.BaseGeometry)).any()]
if geom_cols:
    print("🧹 Tar bort extra geometri-kolumner:", geom_cols)
    gdf_clean = gdf_clean.drop(columns=geom_cols, errors="ignore")

# Skapa Folium-lager utan att försöka serialisera själva geometry-kolumnen
stad_layer = folium.FeatureGroup(name="🟠 Toaletter (Stockholms stad)")

for _, row in gdf_clean.iterrows():
    if pd.isna(row["lat"]) or pd.isna(row["lon"]):
        continue

    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=8,
        fill=True,
        fill_color="#ff6600",
        color="#cc5500",
        weight=2,
        fill_opacity=0.7,
        popup=folium.Popup(build_toilet_popup(row), max_width=350),
    ).add_to(stad_layer)


stad_layer.add_to(m)

gdf_jc = gdf_clean[
    gdf_clean["Driftansvar"]
    .astype(str)
    .str.strip()
    .eq("JCDecaux")
].copy()

jcdecaux_layer = folium.FeatureGroup(
    name="🟣 Endast Stockholm stad JCDecaux (uppsagt avtal)",
    show=False
)

for _, row in gdf_jc.iterrows():
    if pd.isna(row["lat"]) or pd.isna(row["lon"]):
        continue

    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=8,                 # större → analytiskt fokus
        fill=True,
        fill_color="#7b2cff",
        color="#4b1a99",
        weight=2,
        fill_opacity=0.85,
        popup=folium.Popup(build_toilet_popup(row), max_width=350),
    ).add_to(jcdecaux_layer)

jcdecaux_layer.add_to(m)


🧹 Tar bort extra geometri-kolumner: ['geometry_centroid']


In [50]:
gdf_jc

,Adress,Anläggningsnummer,Antal_WC,Antal_urinoarer,Beskrivning,Bild,CHANGE_DATE,CID,CREATE_DATE,Driftansvar,...,Vattenservice,fid,FEATURE_TYPE_NAME,geometry,lon,lat,CHANGE_DATE_fmt,CREATE_DATE_fmt,CREATE_DATE_DT,CHANGE_DATE_DT
1,Norr Mälarstrand 3 - kajen,G60,NaN,NaN,"Tillgänglighetsanpassad WC, 2 x WC och 1 x uri...",None,2.024012e+11,1.538073e+15,2.018023e+11,JCDecaux,...,None,2,Toalett,POINT (152988.491 6579085.031),18.052495,59.326889,None,None,2018-02-28 21:13:00,2024-01-17 08:44:00
3,Sveavägen 39A,G67,NaN,NaN,Tillgänglighetsanpassad,None,2.024012e+11,1.538074e+15,2.018023e+11,JCDecaux,...,None,4,Toalett,POINT (153474.156 6580313.629),18.061045,59.337914,None,None,2018-02-28 21:13:00,2024-01-17 08:44:00
4,Norra Bantorget - Vasagatan 59,G14,1.0,NaN,Tillgänglighetsanpassad WC,None,2.024012e+11,1.530323e+15,2.018011e+11,JCDecaux,...,None,5,Toalett,POINT (153148.155 6579983.768),18.055312,59.334955,None,None,2018-01-08 16:57:00,2024-01-17 08:45:00
5,Eriksdalsbadet,G40,NaN,NaN,"Tillgänglighetsanpassad WC, 2 x WC, 1 x urinoar",None,2.024012e+11,1.530326e+15,2.018011e+11,JCDecaux,...,None,6,Toalett,POINT (154032.4 6576561.095),18.070784,59.304224,None,None,2018-01-08 16:57:00,2024-01-17 08:45:00
6,Bagarmossen C t-bana,G17,NaN,NaN,Tillgänglighetsanpassad WC,None,2.024012e+11,1.530323e+15,2.018011e+11,JCDecaux,...,None,7,Toalett,POINT (157466.297 6573447.361),18.130955,59.276226,None,None,2018-01-08 16:57:00,2024-01-17 08:45:00
7,Sätradalsparken - Lövsätragränd 42,G43,NaN,NaN,Tillgänglighetsanpassad WC,None,2.024101e+11,1.530326e+15,2.018011e+11,JCDecaux,...,None,8,Toalett,POINT (145221.191 6574491.223),17.916159,59.285636,None,None,2018-01-08 16:57:00,2024-10-10 12:22:00
8,Sankt Eriksplan 6C,G33,NaN,NaN,Tillgänglighetsanpassad WC,None,2.024012e+11,1.530325e+15,2.018011e+11,JCDecaux,...,None,9,Toalett,POINT (152121.665 6580505.571),18.037282,59.339646,None,None,2018-01-08 16:57:00,2024-01-17 08:46:00
9,Västerholmsparken,G39,NaN,NaN,Tillgänglighetsanpassad WC,None,2.024012e+11,1.530326e+15,2.018011e+11,JCDecaux,...,None,10,Toalett,POINT (144232.636 6572925.339),17.898857,59.271567,None,None,2018-01-08 16:57:00,2024-01-17 08:47:00
10,Wollmar Yxkullsgatan - Rosenlundsgatan,G56,NaN,NaN,Tillgänglighetsanpassad,None,2.024012e+11,1.538072e+15,2.018023e+11,JCDecaux,...,None,11,Toalett,POINT (153217.098 6577881.611),18.056492,59.316085,None,None,2018-02-28 21:13:00,2024-01-17 08:47:00
11,Tessinparken - De Geersgatan 13,G22,NaN,NaN,Tillgänglighetsanpassad WC,None,2.024012e+11,1.530324e+15,2.018011e+11,JCDecaux,...,None,12,Toalett,POINT (155460.777 6580909.08),18.095968,59.343238,None,None,2018-01-08 16:57:00,2024-01-17 08:47:00


In [51]:
from datetime import datetime, timezone

generated_at = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M UTC")


### Add OSM to map layer OSM

2026-01-23: 'node': 181, 'way': 56}

In [52]:
# should be 237 
rendered = []

for el in osm_data["elements"]:
    lat, lon = get_lat_lon(el)
    if lat is None:
        continue
    rendered.append(el)

len(rendered)


254

In [53]:
#assert len(rendered) == 240, f"Expected 240 toilets, rendered {len(rendered)}"

In [54]:
from datetime import date, datetime

STALE_YEARS = 3

def parse_osm_date(value):
    try:
        return datetime.strptime(value, "%Y-%m-%d").date()
    except Exception:
        return None


def classify_toilet(tags: dict) -> str:
    """
    Returns one of: fixme, stale, unknown, ok
    """
    if "fixme" in tags:
        return "fixme"

    cd = tags.get("check_date")
    if not cd:
        return "unknown"

    parsed = parse_osm_date(cd)
    if not parsed:
        return "unknown"

    age_years = date.today().year - parsed.year
    if age_years >= STALE_YEARS:
        return "stale"

    return "ok"


In [55]:
def classify_toilet_lifecycle(tags: dict) -> str:
    if tags.get("disused:amenity") == "toilets":
        return "disused"
    if tags.get("amenity") == "toilets":
        return "active"
    return "other"

In [56]:
def marker_color(tags: dict) -> str:
    if tags.get("disused:amenity") == "toilets":
        return "red"
    if tags.get("amenity") == "toilets":
        return "green"
    return "gray"
    

In [57]:
import folium

elements = osm_data.get("elements", [])

fg_active = folium.FeatureGroup(
    name="🚻 Open Street Map Toaletter – aktiva",
    show=True
)

fg_disused = folium.FeatureGroup(
    name="🚫 Open Street Map Toaletter – avstängda",
    show=True
)

for el in osm_data["elements"]:
    lat, lon = get_lat_lon(el)
    if lat is None or lon is None:
        continue

    tags = el.get("tags", {})

    if lat is None or lon is None:
        continue
    lifecycle = classify_toilet_lifecycle(tags)
    colorMarker = marker_color(tags)
    popup_html = build_osm_toilet_popup(el, lat=lat, lon=lon,
                    lifecycle=lifecycle)
    
    marker = folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(popup_html, max_width=380),
        icon=folium.Icon(
            icon="restroom",
            prefix="fa",
            color=colorMarker,
        ),
    )
    if lifecycle == "active":
        fg_active.add_child(marker)
    elif lifecycle == "disused":
        fg_disused.add_child(marker)

fg_active.add_to(m)
fg_disused.add_to(m)




### Add legend and Sthlm

In [58]:
legend_html = f"""
<div style="
    position: fixed;
    bottom: 30px;
    left: 30px;
    width: 320px;
    z-index: 10000;
    font-size: 12px;
">

  <!-- Header / Toggle -->
  <div onclick="var c=document.getElementById('legend-content');
                c.style.display = (c.style.display === 'none') ? 'block' : 'none';"
       style="
        cursor: pointer;
        background: white;
        padding: 8px;
        border: 2px solid #ccc;
        border-radius: 6px 6px 0 0;
        font-weight: bold;
       ">
    🟠 Offentliga toaletter – Stockholms stad v{Mapversion}
    <span style="float:right;">▼</span>
  </div>

  <!-- Collapsible content -->
  <div id="legend-content" style="
        display: none;
        background: white;
        padding: 10px;
        border: 2px solid #ccc;
        border-top: none;
        border-radius: 0 0 6px 6px;
  ">

    <small><i>Karta genererad: {generated_at}</i></small><br><br>

    <b>Syfte</b><br>
    Datat används för att verifiera och förbättra OSM.  
    Stockholms stad gör om en hel del – se issue
    (<i>amenity=toilets</i>).<br><br>

    <b>Mer info</b><br>
    • <a href="https://github.com/salgo60/Offentliga-toaletter/issues/1" target="_blank">
      GitHub Issue #1
    </a><br>
    • <a href="https://dataportalen.stockholm.se/dataportalen/GetMetaDataById?id=LvFeature15303196&showmetadataview" target="_blank">
      Stockholms stads öppna data
    </a>

  </div>
</div>
"""


In [59]:
#legend_html

In [60]:
m.get_root().html.add_child(folium.Element(legend_html))
folium.LayerControl(collapsed=False).add_to(m)

In [61]:
from branca.element import MacroElement
from jinja2 import Template

class MinPlatsControl(MacroElement):
    _template = Template("""
        {% macro script(this, kwargs) %}
        var minPlats = L.control({position: 'bottomright'});

        minPlats.onAdd = function (map) {
            var div = L.DomUtil.create('div', 'min-plats-control');
            div.innerHTML = '📍 <strong>Min plats</strong>';

            // Gör klickbar
            L.DomEvent.disableClickPropagation(div);
            L.DomEvent.on(div, 'click', function (e) {
                map.locate({setView: true, maxZoom: 16});
            });

            return div;
        };

        minPlats.addTo({{this._parent.get_name()}});
        {% endmacro %}
    """)

m.add_child(MinPlatsControl())
from folium import Element

css = """
<style>
.min-plats-control {
    cursor: pointer;
    background: #cfe8ef;
    padding: 8px 12px;
    border-radius: 6px;
    box-shadow: 0 0 6px rgba(0,0,0,0.3);
    font-size: 16px;
    user-select: none;
}
.min-plats-control:hover {
    background: #b9dde7;
}
</style>
"""
m.get_root().html.add_child(Element(css))


In [62]:
m

In [63]:
# Exportera kartan https://salgo60.github.io/Offentliga-toaletter/Notebook/Toaletter_sthlm.html
html_map_file = "Toaletter_sthml_" + Mapversion.replace(".", "_") + ".html"
m.save(html_map_file) 
m.save("Toaletter_sthlm_latest.html") 

In [64]:
html_map_file

'Toaletter_sthml_0_7.html'

In [65]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

Date: 2026-01-30 02:04:43
Total time elapsed: 3.54 seconds
